# Récupération jeu de données

L'objectif de ce notebook

Pour la documentation api Spotify



*   https://spotifypy.readthedocs.io/en/latest/api.html
*   https://github.com/spotipy-dev/spotipy/blob/master/examples/search.py
*   https://pypi.org/project/spotipy/








## Installation de l'api SPOTIFY

In [22]:
 # Installation des packages pour Spotify
#pip install spotipy 

## Import des librairies

In [23]:
from function.utils import tqdm, glob, json, os
from function.utils import requests, BeautifulSoup #TODO : ligne a supprimer.

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Récupération des tokens

/!\ Attention /!\  
Pour pouvoir récuperer les access tokens de spotify il est impératif d'avoir un compte free ou premium.  
De plus avoir créer son APP.  
Par la suite, vous obtiendrez les client_id et client_id_secret permettant l'accès à l'API et le bon fonctionnement du code.

In [24]:
from function.function import get_token

PATH_TOKEN = "../data/token/*"
PATH_JSON = "../data/json_file/"
PATH_TXT = "../data/txt/"

liste_txt_file = glob.glob(PATH_TOKEN)

GENIUS_ACCESS_TOKEN = None
SPOTIFY_client_id = None
SPOTIFY_client_secret = None

GENIUS_ACCESS_TOKEN, SPOTIFY_client_id, SPOTIFY_client_secret = get_token(liste_txt_file)


In [25]:
from function.dictionnary import get_dictionnary

dictionnaire_song = get_dictionnary(PATH_JSON, "dictionnaire_song2.json")
liste_artiste = get_dictionnary(PATH_JSON, "dictionnaire_artiste.json")
data_artiste_titre = get_dictionnary(PATH_JSON, "data_artiste_titre.json")

#identifiant spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIFY_client_id,
                                                           client_secret=SPOTIFY_client_secret))

### Pour un artiste dans la liste on met à jour les titres qu'il à réalisé.

In [26]:
set_artistes_existants = {artiste[1] for artiste in data_artiste_titre}

for nom_artiste in tqdm.tqdm(liste_artiste):
    if nom_artiste not in set_artistes_existants:
        results = sp.search(q=nom_artiste, limit=50) # requete de la recherche
        if 'tracks' in results and 'items' in results['tracks']:
            Info_titre = [(len(data_artiste_titre) + idx, nom_artiste, data['name']) for idx, data in enumerate(results['tracks']['items']) ]
        data_artiste_titre.extend(Info_titre)
        set_artistes_existants.add(nom_artiste)

             
with open(PATH_JSON+"data_artiste_titre.json", "w", encoding="utf-8") as json_file:
    json.dump(data_artiste_titre, json_file, indent=4, ensure_ascii=False)


100%|██████████| 3486/3486 [00:35<00:00, 97.66it/s]


In [ ]:
from function.dictionnary import update_dictionary

headers = {"User-Agent": "Mozilla/5.0"}

if os.path.exists("checkpoint.txt"):
    with open("checkpoint.txt", "r") as f:
        start_index = int(f.read().strip())
else:
    start_index = 0

save_every = 1000

[[0, 'Sandie Shaw', 'Puppet On A String'],
 [1, 'Sandie Shaw', 'Sandy'],
 [2, 'Sandie Shaw', "(There's) Always Something There To Remind Me"],
 [3, 'Sandie Shaw', 'Pourvu Que Ça Dure'],
 [4, 'Sandie Shaw', 'Those Were The Days'],
 [5, 'Sandie Shaw', 'Love Me Do'],
 [6, 'Sandie Shaw', 'Love Me, Please Love Me'],
 [7, 'Sandie Shaw', 'Le Temps Des Fleurs'],
 [8, 'Sandie Shaw', 'Monsieur Dupont'],
 [9, 'Sandie Shaw', 'Long Live Love - Mono Mix'],
 [10, 'Sandie Shaw', 'Qué Tiempo Tan Feliz'],
 [11, 'Sandie Shaw', "Girl Don't Come"],
 [12, 'Sandie Shaw', 'Message Understood'],
 [13, 'Sandie Shaw', "Girl Don't Come"],
 [14, 'Sandie Shaw', 'Mama Roux'],
 [15, 'Sandie Shaw', 'Un tout petit pantin'],
 [16, 'Sandie Shaw', 'Love Me, Please Love Me - Italian Version'],
 [17, 'Sandie Shaw', 'Sympathy For The Devil'],
 [18, 'Sandie Shaw', 'Watch the Dog That Bring the Bone'],
 [19, 'Sandie Shaw', 'Tell the Boys'],
 [20, 'Sandie Shaw', 'Reviewing The Situation'],
 [21, 'Sandie Shaw', 'Puppet On A Stri

In [ ]:
#TODO : Dans le scrapping il récupère aussi des livres     
for idx in tqdm.tqdm(range(start_index, len(data_artiste_titre))):
    artist_name, song_title = data_artiste_titre[idx][1:]
    try:
        
        if idx % save_every == 0:
            with open("checkpoint.txt", "w") as w:
                w.write(str(idx))

        # Si l'artiste est dans le dictionnaire et que le titre est présent
        if artist_name in dictionnaire_song and song_title in dictionnaire_song[artist_name].keys():

            # Si les paroles sont None ou vide tu mets à jour
            if dictionnaire_song[artist_name][song_title]["lyrics_primaire"] == None or dictionnaire_song[artist_name][song_title]["lyrics_primaire"] == "":
                dictionnaire_song = update_dictionary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)
            # Sinon tu passes
            continue
        else:
            dictionnaire_song = update_dictionary(song_title, artist_name, GENIUS_ACCESS_TOKEN, dictionnaire_song, headers)

    except Exception as e:
        print(e)
        #print(f"erreur à l'index {idx} : {e}")

        with open("checkpoint.txt", "w") as w:
            w.write(str(idx))
        break


  2%|▏         | 3282/165963 [4:25:12<240:43:54,  5.33s/it]

Erreur pendant la requête Genius : Expecting value: line 1 column 1 (char 0)


  6%|▌         | 10190/165963 [13:28:13<60:46:46,  1.40s/it] 

In [ ]:
with open(PATH_JSON+"dictionnaire_song2.json", "w") as json_file:
    json.dump(dictionnaire_song, json_file, indent=4)